# Install and import required libraries

In [16]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import google.colab
from tqdm import tqdm
import re
import warnings
import csv

from flair.datasets import UD_INDONESIAN
from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.data import Sentence

warnings.filterwarnings("ignore")

# Import data

In [18]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [19]:
df_tweets = pd.read_csv('/content/drive/MyDrive/TA/tweets/ruhutsitompul/done_labelled_ruhutsitompul.csv', delimiter=',', error_bad_lines=False)
df_tweets

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,...,clean_tweets_len,Month,social,historical,dehumanization,accusation,attack,loyalty,threat,Tweets.1
0,0,1585739644011892736,2022-10-27 21:06:07,1585739644011892736,460,80,Sama banget Komentar ini dgn para pendukung se...,Sama banget Komentar ini dgn para pendukung se...,NaN,NaN,...,11,October,1,0,0,0,0.0,0,0,Sama banget Komentar ini dgn para pendukung se...
1,1,1585737538806853632,2022-10-27 20:57:45,1585737538806853632,420,92,Ha ha ha sigundul penguasa ancol karena selama...,Ha ha ha sigundul penguasa ancol karena selama...,NaN,NaN,...,15,October,1,0,0,1,0.0,0,0,Ha ha ha sigundul penguasa ancol karena selama...
2,2,1584780616369049605,2022-10-25 05:35:17,1584780616369049605,343,40,Kasihan Ibu ini jadi korban akibat dicuci otak...,Kasihan Ibu ini jadi korban akibat dicuci otak...,NaN,NaN,...,14,October,0,0,1,1,0.0,0,0,Kasihan Ibu ini jadi korban akibat dicuci otak...
3,3,1584778048120836096,2022-10-25 05:25:05,1584778048120836096,212,23,Ha ha hakadrun pada stresssssss mengenai beber...,Ha ha hakadrun pada stresssssss mengenai beber...,NaN,NaN,...,18,October,1,0,1,0,0.0,0,0,Ha ha hakadrun pada stresssssss mengenai beber...
4,4,1584774254502912000,2022-10-25 05:10:00,1584774254502912000,860,63,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,NaN,NaN,...,33,October,0,0,1,0,0.0,0,0,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4178,4178,1118666888647602177,2019-04-18 00:05:44,1118666888647602177,0,0,@NediSetiadi Kami sedih krn biar sudah kalian ...,@NediSetiadi Kami sedih krn biar sudah kalian ...,NaN,NaN,...,14,April,1,0,0,1,0.0,0,0,@NediSetiadi Kami sedih krn biar sudah kalian ...
4179,4179,1118665503642247170,2019-04-18 00:00:14,1118665503642247170,7,0,@TanYoana haiya Amoy ini makin stresssssss kac...,@TanYoana haiya Amoy ini makin stresssssss kac...,NaN,NaN,...,5,April,0,0,0,0,0.0,0,0,@TanYoana haiya Amoy ini makin stresssssss kac...
4180,4180,1118664923301572608,2019-04-17 23:57:56,1118664923301572608,1684,200,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...,NaN,NaN,...,31,April,0,0,0,0,0.0,0,0,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...
4181,4181,1118396091148460032,2019-04-17 06:09:41,1118396091148460032,6,0,"@panca66 Ha ha ha maaf memilih diri sendiri, s...","@panca66 Ha ha ha maaf memilih diri sendiri, s...",NaN,NaN,...,5,April,0,0,0,0,0.0,0,0,"@panca66 Ha ha ha maaf memilih diri sendiri, s..."


In [20]:
# Remove tweets without any aspect
df_tweets = df_tweets.loc[(df_tweets[['social', 'historical', 'dehumanization', 'accusation', 'attack', 'loyalty', 'threat']] != 0).any(axis=1)]
df_tweets = df_tweets.reset_index(drop=True)
df_tweets

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,...,clean_tweets_len,Month,social,historical,dehumanization,accusation,attack,loyalty,threat,Tweets.1
0,0,1585739644011892736,2022-10-27 21:06:07,1585739644011892736,460,80,Sama banget Komentar ini dgn para pendukung se...,Sama banget Komentar ini dgn para pendukung se...,NaN,NaN,...,11,October,1,0,0,0,0.0,0,0,Sama banget Komentar ini dgn para pendukung se...
1,1,1585737538806853632,2022-10-27 20:57:45,1585737538806853632,420,92,Ha ha ha sigundul penguasa ancol karena selama...,Ha ha ha sigundul penguasa ancol karena selama...,NaN,NaN,...,15,October,1,0,0,1,0.0,0,0,Ha ha ha sigundul penguasa ancol karena selama...
2,2,1584780616369049605,2022-10-25 05:35:17,1584780616369049605,343,40,Kasihan Ibu ini jadi korban akibat dicuci otak...,Kasihan Ibu ini jadi korban akibat dicuci otak...,NaN,NaN,...,14,October,0,0,1,1,0.0,0,0,Kasihan Ibu ini jadi korban akibat dicuci otak...
3,3,1584778048120836096,2022-10-25 05:25:05,1584778048120836096,212,23,Ha ha hakadrun pada stresssssss mengenai beber...,Ha ha hakadrun pada stresssssss mengenai beber...,NaN,NaN,...,18,October,1,0,1,0,0.0,0,0,Ha ha hakadrun pada stresssssss mengenai beber...
4,4,1584774254502912000,2022-10-25 05:10:00,1584774254502912000,860,63,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,NaN,NaN,...,33,October,0,0,1,0,0.0,0,0,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,4151,1120875995962101760,2019-04-24 02:23:56,1120875995962101760,26,4,"@ustadtengkuzul Belajarlah menerima kekalahan,...","@ustadtengkuzul Belajarlah menerima kekalahan,...",NaN,NaN,...,12,April,1,0,0,1,0.0,0,0,"@ustadtengkuzul Belajarlah menerima kekalahan,..."
1660,4153,1120574450410217472,2019-04-23 06:25:42,1120574450410217472,1660,264,Aneh tapi Nyata ini bukan HOAX di Uk...,Aneh tapi Nyata ini bukan HOAX di Uk...,NaN,NaN,...,13,April,0,0,0,1,0.0,0,0,Aneh tapi Nyata ini bukan HOAX di Uk...
1661,4166,1119465028040527872,2019-04-20 04:57:15,1119465028040527872,1311,193,"Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr...","Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr...",NaN,NaN,...,24,April,1,0,0,1,0.0,0,0,"Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr..."
1662,4168,1119429279811428353,2019-04-20 02:35:12,1119429279811428353,10,0,@FerdinandHutah2 Masih saja kau menggonggong s...,@FerdinandHutah2 Masih saja kau menggonggong s...,NaN,NaN,...,6,April,0,0,1,0,0.0,0,0,@FerdinandHutah2 Masih saja kau menggonggong s...


# POS tagging with flair

In [21]:
# 1. get the corpus
corpus = UD_INDONESIAN().downsample(0.1)
print(corpus)

# 2. what label do we want to predict?
label_type = 'upos'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

# 4. initialize embeddings
embedding_types = [

    WordEmbeddings('id-crawl'),
    WordEmbeddings('id'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-upos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

2023-03-08 09:53:33,625 Reading data from /root/.flair/datasets/ud_indonesian
2023-03-08 09:53:33,630 Train: /root/.flair/datasets/ud_indonesian/id_gsd-ud-train.conllu
2023-03-08 09:53:33,632 Dev: /root/.flair/datasets/ud_indonesian/id_gsd-ud-dev.conllu
2023-03-08 09:53:33,634 Test: /root/.flair/datasets/ud_indonesian/id_gsd-ud-test.conllu
2023-03-08 09:53:48,724 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-03-08 09:53:49,823 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-03-08 09:53:54,525 Warning: An empty Sentence was created! Are there empty strings in your dataset?
Corpus: 448 train + 56 dev + 56 test sentences
2023-03-08 09:53:54,541 Computing label dictionary. Progress:


448it [00:00, 4657.29it/s]

2023-03-08 09:53:54,694 Dictionary created for label 'upos' with 18 values: NOUN (seen 2138 times), PROPN (seen 1813 times), PUNCT (seen 1398 times), VERB (seen 1031 times), ADP (seen 938 times), PRON (seen 530 times), ADJ (seen 375 times), NUM (seen 333 times), DET (seen 310 times), CCONJ (seen 283 times), ADV (seen 282 times), AUX (seen 210 times), SCONJ (seen 173 times), PART (seen 79 times), X (seen 23 times), SYM (seen 14 times), INTJ (seen 1 times)
Dictionary with 18 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, CCONJ, ADV, AUX, SCONJ, PART, X, SYM, INTJ


2023-03-08 09:54:23,591 SequenceTagger predicts: Dictionary with 18 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, CCONJ, ADV, AUX, SCONJ, PART, X, SYM, INTJ
2023-03-08 09:54:23,717 ----------------------------------------------------------------------------------------------------
2023-03-08 09:54:23,720 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'id-crawl'
      (embedding): Embedding(1000000, 300)
    )
    (list_embedding_1): WordEmbeddings(
      'id'
      (embedding): Embedding(300686, 300)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=600, out_features=600, bias=True)
  (rnn): LSTM(600, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=20, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2023-03-08 09:54:23,722 --------------------------------------------------------

100%|██████████| 2/2 [00:00<00:00,  2.46it/s]

...

2023-03-08 09:58:28,197 ----------------------------------------------------------------------------------------------------
2023-03-08 09:58:35,152 SequenceTagger predicts: Dictionary with 20 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, CCONJ, ADV, AUX, SCONJ, PART, X, SYM, INTJ, <START>, <STOP>


100%|██████████| 2/2 [00:00<00:00,  5.07it/s]

2023-03-08 09:58:36,673 Evaluating as a multi-label problem: False


2023-03-08 09:58:36,711 0.7886	0.7886	0.7886	0.7886
2023-03-08 09:58:36,713 
Results:
- F-score (micro) 0.7886
- F-score (macro) 0.6445
- Accuracy 0.7886

By class:
              precision    recall  f1-score   support

       PROPN     0.7429    0.9579    0.8368       190
        NOUN     0.8494    0.6714    0.7500       210
       PUNCT     0.9854    1.0000    0.9926       135
        VERB     0.7760    0.9238    0.8435       105
         ADP     0.7304    0.9231    0.8155        91
        PRON     0.7500    0.9344    0.8321        61
         NUM     0.8125    0.8966    0.8525        29
         ADJ     0.8571    0.2667    0.4068        45
         ADV     0.1852    0.1724    0.1786        29
       CCONJ     1.0000    0.7857    0.8800        28
         AUX     0.5714    0.5455    0.5581        22
         DET     0.9167    0.3793    0.5366        29
       SCONJ     0.5000    0.0526    0.0952        19
        PART     0.5000    0.4000    0.4444         5

    accuracy           

{'test_score': 0.7885771543086172,
 'dev_score_history': [0.23629893238434163,
  0.4903914590747331,
  0.5722419928825623,
  0.606405693950178,
  0.6604982206405694,
  0.6854092526690392,
  0.706049822064057,
  0.7580071174377224,
  0.7750889679715303,
  0.7836298932384341],
 'train_loss_history': [2.6291418969601184,
  1.9866362692618127,
  1.5969153221682737,
  1.4051699155815753,
  1.2626137192330202,
  1.1160078270535445,
  1.024502293363269,
  0.9353549992998565,
  0.8715583325728778,
  0.8124591664909753],
 'dev_loss_history': [2.2096564769744873,
  1.6100496053695679,
  1.3249773979187012,
  1.1911381483078003,
  1.0377202033996582,
  0.9188407063484192,
  0.8484406471252441,
  0.7241466641426086,
  0.6458045840263367,
  0.6197919249534607]}

#1-gram

In [22]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# specify the category for which to extract the most frequent POS tag patterns
dehumanization_tweets = df_tweets.loc[df_tweets.dehumanization== 1]

# create an empty dictionary to store the POS tag patterns and their frequency
tag_patterns = {}

# iterate over each tweet in the dataset
for tweet in dehumanization_tweets['clean_tweets'] :
    # create a Flair sentence object from the tweet text
    sentence = Sentence(tweet, use_tokenizer=True)
    
    # use the POS tagger to predict the POS tags for the sentence
    tagger.predict(sentence)
    
    # iterate over each token in the sentence and extract the POS tag
    for token in sentence:
        tag = token.get_label().value
        
        # add the POS tag to the tag_patterns dictionary and increment its frequency count
        if tag not in tag_patterns:
            tag_patterns[tag] = 1
        else:
            tag_patterns[tag] += 1

# sort the tag_patterns dictionary by frequency count in descending order
sorted_tag_patterns = {k: v for k, v in sorted(tag_patterns.items(), key=lambda item: item[1], reverse=True)}

# print the top 10 most frequent POS tag patterns
print('Top 10 most frequent POS tag patterns for Dehumanization aspect:')
for tag, freq in list(sorted_tag_patterns.items())[:10]:
    print(f'{tag}: {freq}')

2023-03-08 09:58:43,258 SequenceTagger predicts: Dictionary with 20 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, CCONJ, ADV, AUX, SCONJ, PART, X, SYM, INTJ, <START>, <STOP>
Top 10 most frequent POS tag patterns for Dehumanization aspect:
PROPN: 4978
NOUN: 1817
VERB: 1359
PUNCT: 500
ADJ: 390
PRON: 321
ADP: 300
ADV: 266
AUX: 113
NUM: 70


In [23]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# specify the category for which to extract the most frequent POS tag patterns
accusation_tweets = df_tweets.loc[df_tweets.accusation== 1]

# create an empty dictionary to store the POS tag patterns and their frequency
tag_patterns = {}

# iterate over each tweet in the dataset
for tweet in accusation_tweets['clean_tweets'] :
    # create a Flair sentence object from the tweet text
    sentence = Sentence(tweet, use_tokenizer=True)
    
    # use the POS tagger to predict the POS tags for the sentence
    tagger.predict(sentence)
    
    # iterate over each token in the sentence and extract the POS tag
    for token in sentence:
        tag = token.get_label().value
        
        # add the POS tag to the tag_patterns dictionary and increment its frequency count
        if tag not in tag_patterns:
            tag_patterns[tag] = 1
        else:
            tag_patterns[tag] += 1

# sort the tag_patterns dictionary by frequency count in descending order
sorted_tag_patterns = {k: v for k, v in sorted(tag_patterns.items(), key=lambda item: item[1], reverse=True)}

# print the top 10 most frequent POS tag patterns
print('Top 10 most frequent POS tag patterns for Accusation aspect:')
for tag, freq in list(sorted_tag_patterns.items())[:10]:
    print(f'{tag}: {freq}')

2023-03-08 09:58:58,237 SequenceTagger predicts: Dictionary with 20 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, CCONJ, ADV, AUX, SCONJ, PART, X, SYM, INTJ, <START>, <STOP>
Top 10 most frequent POS tag patterns for Accusation aspect:
PROPN: 3771
NOUN: 1285
VERB: 1168
ADJ: 256
PUNCT: 214
ADP: 189
ADV: 183
PRON: 150
AUX: 112
NUM: 32


In [24]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# specify the category for which to extract the most frequent POS tag patterns
attack_tweets = df_tweets.loc[df_tweets.attack== 1]

# create an empty dictionary to store the POS tag patterns and their frequency
tag_patterns = {}

# iterate over each tweet in the dataset
for tweet in attack_tweets['clean_tweets'] :
    # create a Flair sentence object from the tweet text
    sentence = Sentence(tweet, use_tokenizer=True)
    
    # use the POS tagger to predict the POS tags for the sentence
    tagger.predict(sentence)
    
    # iterate over each token in the sentence and extract the POS tag
    for token in sentence:
        tag = token.get_label().value
        
        # add the POS tag to the tag_patterns dictionary and increment its frequency count
        if tag not in tag_patterns:
            tag_patterns[tag] = 1
        else:
            tag_patterns[tag] += 1

# sort the tag_patterns dictionary by frequency count in descending order
sorted_tag_patterns = {k: v for k, v in sorted(tag_patterns.items(), key=lambda item: item[1], reverse=True)}

# print the top 10 most frequent POS tag patterns
print('Top 10 most frequent POS tag patterns for Attack aspect:')
for tag, freq in list(sorted_tag_patterns.items())[:10]:
    print(f'{tag}: {freq}')

2023-03-08 09:59:09,891 SequenceTagger predicts: Dictionary with 20 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, CCONJ, ADV, AUX, SCONJ, PART, X, SYM, INTJ, <START>, <STOP>
Top 10 most frequent POS tag patterns for Attack aspect:
PROPN: 133
NOUN: 108
VERB: 86
ADJ: 22
PRON: 15
ADV: 14
ADP: 12
AUX: 9
PUNCT: 8
PART: 1


In [25]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# specify the category for which to extract the most frequent POS tag patterns
loyalty_tweets = df_tweets.loc[df_tweets.loyalty== 1]

# create an empty dictionary to store the POS tag patterns and their frequency
tag_patterns = {}

# iterate over each tweet in the dataset
for tweet in loyalty_tweets['clean_tweets'] :
    # create a Flair sentence object from the tweet text
    sentence = Sentence(tweet, use_tokenizer=True)
    
    # use the POS tagger to predict the POS tags for the sentence
    tagger.predict(sentence)
    
    # iterate over each token in the sentence and extract the POS tag
    for token in sentence:
        tag = token.get_label().value
        
        # add the POS tag to the tag_patterns dictionary and increment its frequency count
        if tag not in tag_patterns:
            tag_patterns[tag] = 1
        else:
            tag_patterns[tag] += 1

# sort the tag_patterns dictionary by frequency count in descending order
sorted_tag_patterns = {k: v for k, v in sorted(tag_patterns.items(), key=lambda item: item[1], reverse=True)}

# print the top 10 most frequent POS tag patterns
print('Top 10 most frequent POS tag patterns for Loyalty aspect:')
for tag, freq in list(sorted_tag_patterns.items())[:10]:
    print(f'{tag}: {freq}')

2023-03-08 09:59:17,777 SequenceTagger predicts: Dictionary with 20 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, CCONJ, ADV, AUX, SCONJ, PART, X, SYM, INTJ, <START>, <STOP>
Top 10 most frequent POS tag patterns for Loyalty aspect:
PROPN: 287
VERB: 90
NOUN: 88
PRON: 17
AUX: 15
ADV: 14
ADP: 12
PUNCT: 11
ADJ: 10
NUM: 5


In [26]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# specify the category for which to extract the most frequent POS tag patterns
threat_tweets = df_tweets.loc[df_tweets.threat== 1]

print(f'Number of tweets in filtered dataframe: {len(df_tweets)}')

# create an empty dictionary to store the POS tag patterns and their frequency
tag_patterns = {}

# iterate over each tweet in the dataset
for tweet in threat_tweets['clean_tweets'] :
    # create a Flair sentence object from the tweet text
    sentence = Sentence(tweet, use_tokenizer=True)
    
    # use the POS tagger to predict the POS tags for the sentence
    tagger.predict(sentence)
    
    # iterate over each token in the sentence and extract the POS tag
    for token in sentence:
        tag = token.get_label().value
        
        # add the POS tag to the tag_patterns dictionary and increment its frequency count
        if tag not in tag_patterns:
            tag_patterns[tag] = 1
        else:
            tag_patterns[tag] += 1

# sort the tag_patterns dictionary by frequency count in descending order
sorted_tag_patterns = {k: v for k, v in sorted(tag_patterns.items(), key=lambda item: item[1], reverse=True)}

# print the top 10 most frequent POS tag patterns
print('Top 10 most frequent POS tag patterns for Threat aspect:')
for tag, freq in list(sorted_tag_patterns.items())[:10]:
    print(f'{tag}: {freq}')

2023-03-08 10:00:13,245 SequenceTagger predicts: Dictionary with 20 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, CCONJ, ADV, AUX, SCONJ, PART, X, SYM, INTJ, <START>, <STOP>
Number of tweets in filtered dataframe: 1664
Top 10 most frequent POS tag patterns for Threat aspect:
PROPN: 472
VERB: 202
NOUN: 193
ADJ: 28
ADP: 27
PRON: 25
PUNCT: 20
AUX: 19
ADV: 15
NUM: 6
